In [ ]:
import csv
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
import scipy
import seaborn as sns
import math
sns.set()
sns.set_style("white")
plt.rcParams.update({'font.size': 20})

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import ConstantKernel, RBF, Matern, RationalQuadratic, ExpSineSquared

In [ ]:
plt.rc('xtick', labelsize=20)    
plt.rc('ytick', labelsize=20)  

In [ ]:
def getData(df):
    # create array of pairs of all of these
    times = np.zeros(len(df))
    data = []
    for i, row in df.iterrows():
        data.append([])
#         for count, joint_idx in enumerate(range(0,3*3,3)):
#             data[i].append(list(row[2+joint_idx:2+joint_idx+3]))
        for count, joint_idx in enumerate(range(0,50*4,4)):
            if row[11+joint_idx+3] <= 0:
                data[i].append([])
            else:
                data[i].append(list(row[11+joint_idx:11+joint_idx+3]))

        times[i] = row[1]
    return data, times

In [ ]:
def get1Ddata(data, times,  marker_idx, coord_idx):
    X = []
    y = []
    
    relevant = np.array(data)[:,marker_idx]
    for i, marker in enumerate(relevant):
        if len(marker) == 0:
            continue
        y.append(marker[coord_idx])
        X.append(times[i])
    y = np.array(y)
    X = np.array(X).reshape(-1,1)
    return X,y

In [ ]:
train_data1, train_times1 = getData(pd.read_csv('data_GP/data_GP/AG/block1-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213203046-59968-right-speed_0.500.csv', delimiter=','))
train_data2, train_times2 = getData(pd.read_csv('data_GP/data_GP/AG/block2-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204004-59968-right-speed_0.500.csv', delimiter=','))
train_data3, train_times3 = getData(pd.read_csv('data_GP/data_GP/AG/block3-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204208-59968-right-speed_0.500.csv', delimiter=','))
train_data4, train_times4  = getData(pd.read_csv('data_GP/data_GP/AG/block4-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213204925-59968-right-speed_0.500.csv', delimiter=','))
test_data1, test_times1   = getData(pd.read_csv('data_GP/data_GP/AG/block5-UNWEIGHTED-SLOW-NONDOMINANT-RANDOM/20161213210121-59968-right-speed_0.500.csv', delimiter=','))

In [ ]:
marker = 20
coord = 0
Xwhole1, ywhole1 = get1Ddata(train_data1, train_times1, marker, coord)
Xwhole2, ywhole2 = get1Ddata(train_data2, train_times2, marker, coord)
Xwhole4, ywhole4 = get1Ddata(train_data3, train_times3, marker, coord)
Xwhole5, ywhole5 = get1Ddata(train_data4, train_times4, marker, coord)
Xwhole3, ywhole3 = get1Ddata(test_data1, test_times1, marker, coord)
plt.clf()
# plt.scatter(Xwhole1,ywhole1)
plt.scatter(Xwhole2,ywhole2)
plt.scatter(Xwhole3, ywhole3)
plt.scatter(Xwhole4, ywhole4)
plt.scatter(Xwhole5, ywhole5)

In [ ]:
def fitAndPlotGP(Xtrain,ytrain,Xtest,ytest, kernel,times, title='', graph=True):
    ytrain = ytrain.copy()
    y_no_noise = ytrain.copy()
#     dy = .05 * np.random.random(y.shape)
#     noise = np.random.normal(0, dy)
#     ytrain += noise
    
    
    
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=0, normalize_y=True)
    gp.fit(Xtrain,ytrain)
    pred, std = gp.predict(Xtest, return_std=True)
    
    testCoef = gp.score(Xtest,ytest)
#     print('Test Coefficient',gp.score(Xtest,ytest))
        
    sq_res = np.sum((ytest-pred)**2)
    pred, std = gp.predict(times, return_std=True)

        
    if graph:
        plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
        plt.plot(Xtest,ytest, colors[2], label='test data', linewidth=5)

        plt.plot(Xtrain, y_no_noise, 'o',color=colors[0], ms=5, label='training points')

        plt.fill_between(times.flat, pred-1.96*std, pred+1.96*std, color=colors[3], alpha=.2)
        plt.plot(times, pred, colors[3], lw=2, label='mean prediction', linewidth=5)
        plt.axis([0, times.flatten().max(), min(ytest.min(),pred.min())-.2, max(ytest.max(),pred.max())+.2])
        plt.title(title+'Test Set Coefficient of Determination: ' + str(testCoef) + '\n' + str(gp.kernel_),fontsize=20)

        plt.xlabel('time',fontsize=25)
        plt.ylabel('coordinate location',fontsize=25)
        plt.legend(loc="best",  scatterpoints=1, prop={'size': 15},  title_fontsize=15)
        return sq_res
    return sq_res, pred, std, gp.kernel_

# Global Kernel Function

In [ ]:
sns.set_palette("pastel")
pal = sns.color_palette("pastel")
colors = pal.as_hex()



marker = 20
coord = 0
# Xwhole1, ywhole1 = get1Ddata(train_data1, train_times1, marker, coord)
Xwhole2, ywhole2 = get1Ddata(train_data2, train_times2, marker, coord)
Xwhole4, ywhole4 = get1Ddata(train_data3, train_times3, marker, coord)
Xwhole5, ywhole5 = get1Ddata(train_data4, train_times4, marker, coord)
Xwhole3, ywhole3 = get1Ddata(test_data1, test_times1, marker, coord)

Xwhole = np.concatenate([Xwhole2,Xwhole3,Xwhole4])
ywhole = np.concatenate([ywhole2,ywhole3,ywhole4])



print(Xwhole.shape)
size = int(len(Xwhole)/16)
# size = 500
# between = np.random.randint(len(Xwhole), size=size)
between = np.random.choice(np.arange(len(Xwhole)), size=size, replace=False)

X = np.array([Xwhole[i] for i in between])
y = np.array([ywhole[i] for i in between])


print(X.shape)

# temporal_X = np.arange(i*skips, i*skips+skips,.05).reshape(-1,1)

# y = np.concatenate([y1,y2])

# kernel = 1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1)+ WhiteKernel() + DotProduct() 
kernel = ConstantKernel() * RBF()+ WhiteKernel() 

last_time = sorted(Xwhole.flat)[-1]
temporal_X = np.arange(0, last_time,.05).reshape(-1,1)
# kernel = RBF()


fitAndPlotGP(X,y,Xwhole5,ywhole5,kernel,temporal_X, title="8_x using global kernel\n")


# Local Kernel Functions

In [ ]:
def getTemporalSlice(start, end, data, times):
    times = times.flatten()
    data = data[times.argsort()]

    times = times[times.argsort()]
    idxs = np.where((start < times) & (times <= end))
    return times[idxs].reshape(-1,1), data[idxs]

def avg_group(A, B):
    df = pd.DataFrame({'A':A, 'B':B})
    res = df.groupby('A').mean()
    keys = np.array(list(df.groupby('A').groups.keys()))
    ans = res.to_numpy().flatten()
    return keys,ans
val1 = np.array([1,1,2,2,2,3,4,5,6,6,6,6],dtype=np.float64)
val2 = np.array([2,3,1,2,3,3,4,5,6,6,3,2],dtype=np.float64)
t1,t2=avg_group(times,preds)
plt.scatter(t1,t2)
print(t1)
print(t2)


In [ ]:
%matplotlib
# kernel = ConstantKernel() + RBF() + WhiteKernel()+ DotProduct()
kernel = ConstantKernel(constant_value=.25, constant_value_bounds=(0,10)) + RBF() + WhiteKernel(noise_level=3.0, noise_level_bounds=(0,10))

last_time = sorted(Xwhole.flat)[-1]
print(last_time)

plotting = True
res_list = []
coef_list = []
# for kernel in kernels:
# plt.ioff()
all_temporal_X = np.arange(0, last_time,.005).reshape(-1,1)
for repeat in range(1):
    for iteration in range(29,30,3):
        windows=50
        skips = last_time/windows
        
        
        delta = .1
        window_size = 1
        
        start = np.arange(0, last_time-2*delta, delta)
        end = np.arange(window_size, last_time+delta, delta)

        
        
        print("repeat number {} with delta {} and window_size {}".format(repeat, delta, window_size))

        preds = np.array([])
        stds = np.array([])
        Xtests = np.array([])
        ytests = np.array([])
        times = np.array([])
        Xs = np.array([])
        ys= np.array([])
        sumSqRes = 0
        hyperparams= np.array(kernel.theta)[np.newaxis,:]
        for i in range(min(len(end), len(start))):
            X,y = getTemporalSlice(start[i],end[i], ywhole,Xwhole)
            size = math.ceil(len(X)/16)
            between = np.random.choice(np.arange(len(X)), size=size, replace=False)
            X = np.array([X[i] for i in between])
            y = np.array([y[i] for i in between])
            if(len(X) == 0):
                continue
            Xtest,ytest = getTemporalSlice(start[i],end[i], ywhole5,Xwhole5)
            if(len(X) == 0 or len(Xtest) == 0):
                continue
#             temporal_X = np.arange(i*skips-delta, i*skips+skips+delta,.05).reshape(-1,1)
            temporal_X, temporalY = getTemporalSlice(start[i],end[i], all_temporal_X, all_temporal_X)

    
    
            res, pred, std, opt_kernel = fitAndPlotGP(X,y,Xtest,ytest,kernel,temporal_X, graph=False)


            sumSqRes += res
            preds = np.concatenate([preds,pred])
            stds = np.concatenate([stds,std])
            times = np.concatenate([times,temporal_X.flat])
            Xs = np.concatenate([Xs,X.flat])
            ys = np.concatenate([ys,y]) 
            hyperparams = np.concatenate([hyperparams, np.array(opt_kernel.theta)[np.newaxis,:]])
        print("Squared Residuals", sumSqRes)
        meanSubtracted = np.sum((ywhole5-np.mean(ywhole5))**2)
        testCoef = (1-sumSqRes/meanSubtracted)
        print("Coefficient of determination:", testCoef)

#         res_list.append([windows, sumSqRes])
#         coef_list.append([windows, testCoef])

        if plotting:
            size = math.ceil(len(Xwhole)/16)
            between = np.random.choice(np.arange(len(Xwhole)), size=size, replace=False)
            testx = np.array([Xwhole[i] for i in between])
            testy = np.array([ywhole[i] for i in between])
            
            newTimes, newPreds = avg_group(times, preds)
            newTimes, newStds = avg_group(times, stds)

            plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
            plt.plot(Xwhole5,ywhole5, colors[2], label='test data', linewidth=5)
            plt.plot(testx, testy, 'o',color=colors[0], ms=5, label='training points')
            plt.fill_between(newTimes.flat, newPreds-1.96*newStds, newPreds+1.96*newStds, color=colors[3], alpha=.2)
            plt.plot(newTimes, newPreds, colors[3], lw=2, label='mean prediction', linewidth=5)
            plt.axis([0, times.flatten().max(), preds.flatten().min()-.5, preds.flatten().max()+.5])
            plt.title('8_x Using Local Kernel with Window Size {}, Delta {}'.format(window_size, delta) ,fontsize=20)
            plt.xlabel('time',fontsize=25)
            plt.ylabel('coordinate location',fontsize=25)
            plt.legend(loc="best",  scatterpoints=1, D)
            
#             for time_splits in range(windows):
#                 plt.axvline(x=time_splits*skips)
#             plt.savefig('{}_{}_{}windows_12percent_sampled.png'.format(marker,coord,windows))
            plt.show()



            def plotParams(hyperparams, param_names, start, end):
                plt.figure(figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
                all_temporal_X = np.arange(0, last_time,.05)
                all_params = []
                for i in range(len(all_temporal_X)):
                    sums = np.zeros(hyperparams.shape[-1])
                    count = 0
                    for window_num in range(min(len(end), len(start))):
#                         print('testing {} with range {} {}'.format(all_temporal_X[i], start[window_num], end[window_num]))

                        if start[window_num] <= all_temporal_X[i] and  all_temporal_X[i] < end[window_num]:
#                             print('in window num {} with range {} {}'.format(window_num, start[window_num], end[window_num]))
                            # check through hyperparams in this window and add them
                            sums += hyperparams[window_num+1]
                            count += 1
                        else:
                            continue
                    all_params.append(sums/count)
                    

                            
#                 for item in range(len(hyperparams[0])):
#                     print(hyperparams[1:,item])
#                     plt.plot(hyperparams[1:,item], label=param_names[item], linewidth=5)
#                 plt.xticks(np.arange(windows,step=1))
                plt.plot(all_temporal_X, np.array(all_params)[:,0], label=param_names[0], linewidth=5)
                plt.plot(all_temporal_X, np.array(all_params)[:,1], label=param_names[1], linewidth=5)
                plt.plot(all_temporal_X, np.array(all_params)[:,2], label=param_names[2], linewidth=5)
                plt.plot(Xwhole5,ywhole5*10, colors[2], label='scaled trajectory', linewidth=5, c=colors[3])
                print(hyperparams[0])
                plt.axhline(hyperparams[0][0],label='init_'+param_names[0], linewidth=5, alpha=.6,c=colors[0])
                plt.axhline(hyperparams[0][1],label='init_'+param_names[1], linewidth=5, alpha=.6,c=colors[1])
                plt.axhline(hyperparams[0][2],label='init_'+param_names[2], linewidth=5, alpha=.6,c=colors[2])
                plt.legend(loc="lower left",  scatterpoints=1, prop={'size': 15},  title_fontsize=15)
                plt.xlabel('time',fontsize=25)
                plt.ylabel('hyperparameter values',fontsize=25)
                plt.title("Marker 8_x Average Hyperparameters Over Time; Window Size {}, Delta {}".format(window_size, delta),fontsize=20)
#                 plt.savefig('{}_{}_{}windows_12percent_sampled_hyperparams.png'.format(marker,coord,windows))

                plt.show()
    
            

            param_names = ['sigma_f', 'sigma_l', 'sigma_n']
            plotParams(hyperparams,param_names, start, end)


In [ ]:
print(hyperparams[1:])

In [ ]:
print(hyperparams[1:])